# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124952e+02     1.710152e+00
 * time: 0.13184905052185059
     1     1.096403e+01     8.397447e-01
 * time: 0.9922900199890137
     2    -1.203016e+01     9.307049e-01
 * time: 1.1307399272918701
     3    -3.416090e+01     7.147939e-01
 * time: 1.2904760837554932
     4    -4.771356e+01     5.316428e-01
 * time: 1.4401469230651855
     5    -5.694208e+01     1.853727e-01
 * time: 1.5880439281463623
     6    -5.982599e+01     1.053656e-01
 * time: 1.6980979442596436
     7    -6.080109e+01     6.201304e-02
 * time: 1.8143188953399658
     8    -6.126609e+01     6.013716e-02
 * time: 1.9233789443969727
     9    -6.160873e+01     3.809630e-02
 * time: 2.0406739711761475
    10    -6.184906e+01     2.619006e-02
 * time: 2.1577789783477783
    11    -6.198924e+01     2.797390e-02
 * time: 2.268151044845581
    12    -6.207044e+01     2.294783e-02
 * time: 2.383380889892578
    13    -6.212427e+01     1.641862e-02
 * time: 2.49255800247

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671074
    AtomicLocal         -18.8557683
    AtomicNonlocal      14.8522642
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485377 
    Xc                  -19.3336822

    total               -62.261666459445
